# Explore Embryoscope Clinic Data

This notebook helps you explore the silver layer for each clinic individually.

In [8]:
import duckdb
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', None)

# List all clinic DBs
db_dir = Path('../../database')
clinic_dbs = sorted([p for p in db_dir.glob('embryoscope_*.db') if 'test' not in str(p) and 'huntington_data_lake' not in str(p)])
# clinic_dbs = sorted([p for p in db_dir.glob('embryoscope_*') if 'test' in str(p) and 'huntington_data_lake' not in str(p)])
clinic_dbs

[WindowsPath('../../database/embryoscope_belo_horizonte.db'),
 WindowsPath('../../database/embryoscope_brasilia.db'),
 WindowsPath('../../database/embryoscope_ibirapuera.db'),
 WindowsPath('../../database/embryoscope_vila_mariana.db')]

In [9]:
# Select a clinic DB to explore
clinic_idx = -1  # Change this index to select a different clinic
db_path = clinic_dbs[clinic_idx]
print(f'Exploring: {db_path}')

Exploring: ..\..\database\embryoscope_vila_mariana.db


In [10]:
# Connect to the selected DB
con = duckdb.connect(str(db_path))


In [11]:
# List all schemas in the database
schemas = con.execute("SELECT schema_name FROM information_schema.schemata").fetchdf()
schemas


,schema_name
0,bronze
1,main
2,silver
3,information_schema
4,main
5,pg_catalog
6,main


In [12]:
# List all tables in the silver schema
use_schema = 'bronze'
use_schema = 'silver'
tables = con.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{use_schema}'").fetchdf()
tables

,table_name
0,embryo_data
1,idascore
2,patients
3,treatments


In [13]:
# Show schema and sample for each table in silver
for table in tables['table_name']:
    print(f'\n=== {table} ===')
    schema = con.execute(f'PRAGMA table_info({use_schema}.{table})').fetchdf()
    display(schema)
    df = con.execute(f'SELECT * FROM {use_schema}.{table} LIMIT 5').fetchdf()
    display(df)
    lines = con.execute(f'SELECT COUNT(*) FROM {use_schema}.{table}').fetchone()
    print(f'Records: {lines}')
con.close()


=== embryo_data ===


,cid,name,type,notnull,dflt_value,pk
0,0,EmbryoID,VARCHAR,False,None,False
1,1,PatientIDx,VARCHAR,False,None,False
2,2,TreatmentName,VARCHAR,False,None,False
3,3,KIDDate,TIMESTAMP_NS,False,None,False
4,4,KIDScore,VARCHAR,False,None,False
...,...,...,...,...,...,...
183,183,_extraction_timestamp,TIMESTAMP_NS,False,None,False
184,184,_location,VARCHAR,False,None,False
185,185,_row_hash,VARCHAR,False,None,False
186,186,_run_id,VARCHAR,False,None,False


,EmbryoID,PatientIDx,TreatmentName,KIDDate,KIDScore,KIDUser,KIDVersion,Description,EmbryoDescriptionID,EmbryoFate,FertilizationMethod,FertilizationTime,InstrumentNumber,Name_BlastExpandLast,Name_Comment,Name_DynamicScore,Name_EVEN2,Name_EVEN4,Name_EVEN8,Name_Ellipse,Name_FRAG2,Name_FRAG2CAT,Name_FRAG4,Name_FRAG8,Name_ICM,Name_Line,Name_MN2Type,Name_MorphologicalGrade,Name_MorphologicalGradeD5,Name_Nuclei2,Name_Nuclei4,Name_Nuclei8,Name_PN,Name_Pulsing,Name_Strings,Name_TE,Name_Text,Name_ZScore,Name_t2,Name_t3,Name_t4,Name_t5,Name_t6,Name_t7,Name_t8,Name_t9,Name_tB,Name_tEB,Name_tHB,Name_tM,Name_tPB2,Name_tPNa,Name_tPNf,Name_tSB,Name_tSC,Position,Time_BlastExpandLast,Time_Comment,Time_DynamicScore,Time_EVEN2,Time_EVEN4,Time_EVEN8,Time_Ellipse,Time_FRAG2,Time_FRAG2CAT,Time_FRAG4,Time_FRAG8,Time_ICM,Time_Line,Time_MN2Type,Time_MorphologicalGrade,Time_MorphologicalGradeD5,Time_Nuclei2,Time_Nuclei4,Time_Nuclei8,Time_PN,Time_Pulsing,Time_Strings,Time_TE,Time_Text,Time_ZScore,Time_t2,Time_t3,Time_t4,Time_t5,Time_t6,Time_t7,Time_t8,Time_t9,Time_tB,Time_tEB,Time_tHB,Time_tM,Time_tPB2,Time_tPNa,Time_tPNf,Time_tSB,Time_tSC,Timestamp_BlastExpandLast,Timestamp_Comment,Timestamp_DynamicScore,Timestamp_EVEN2,Timestamp_EVEN4,Timestamp_EVEN8,Timestamp_Ellipse,Timestamp_FRAG2,Timestamp_FRAG2CAT,Timestamp_FRAG4,Timestamp_FRAG8,Timestamp_ICM,Timestamp_Line,Timestamp_MN2Type,Timestamp_MorphologicalGrade,Timestamp_MorphologicalGradeD5,Timestamp_Nuclei2,Timestamp_Nuclei4,Timestamp_Nuclei8,Timestamp_PN,Timestamp_Pulsing,Timestamp_Strings,Timestamp_TE,Timestamp_Text,Timestamp_ZScore,Timestamp_t2,Timestamp_t3,Timestamp_t4,Timestamp_t5,Timestamp_t6,Timestamp_t7,Timestamp_t8,Timestamp_t9,Timestamp_tB,Timestamp_tEB,Timestamp_tHB,Timestamp_tM,Timestamp_tPB2,Timestamp_tPNa,Timestamp_tPNf,Timestamp_tSB,Timestamp_tSC,Value_BlastExpandLast,Value_Comment,Value_DynamicScore,Value_EVEN2,Value_EVEN4,Value_EVEN8,Value_Ellipse,Value_FRAG2,Value_FRAG2CAT,Value_FRAG4,Value_FRAG8,Value_ICM,Value_Line,Value_MN2Type,Value_MorphologicalGrade,Value_MorphologicalGradeD5,Value_Nuclei2,Value_Nuclei4,Value_Nuclei8,Value_PN,Value_Pulsing,Value_Strings,Value_TE,Value_Text,Value_ZScore,Value_t2,Value_t3,Value_t4,Value_t5,Value_t6,Value_t7,Value_t8,Value_t9,Value_tB,Value_tEB,Value_tHB,Value_tM,Value_tPB2,Value_tPNa,Value_tPNf,Value_tSB,Value_tSC,WellNumber,_extraction_timestamp,_location,_row_hash,_run_id,embryo_number
0,D2019.06.10_S00009_I3253_P-1,PC10T4L741253_43626.4449943171,05.06.2019,2020-02-13,NA,ADMIN,KIDScoreD5 v3,None,AA1,Unknown,None,2019-06-10 09:00:00,3253,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,t8,None,None,None,None,None,tPB2,tPNa,tPNf,None,None,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,28.2,28.2,28.2,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019.06.10 10:58:10,None,None,None,None,None,2019.06.11 13:14:24,2019.06.11 13:14:24,2019.06.11 13:14:24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,43626.4570676273,None,None,None,None,None,43627.5516752546,43627.5516752546,43627.5516752546,None,None,1,2025-07-15 19:21:41.460065,Vila Mariana,278c0046f8670f26452f4e510ac40220,5a1a0509-7c67-43d2-a882-27ef8fcbf469,1
1,D2019.06.10_S00010_I3253_P-1,PC10T4L741253_43626.4518250926,05.06.2019,2020-02-13,NA,ADMIN,KIDScoreD5 v3,None,AA1,Freeze,None,2019-06-10 09:00:00,3253,None,None,None,None,None,None,None,None,None,None,None,ICM,None,None,None,None,None,None,None,None,None,None,TE,None,None,t2,t3,t4,t5,t6,t7,t8,t9,tB,None,None,tM,None,None,None,tSB,tSC,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3,NaN,NaN,NaN,NaN,NaN,NaN,N

Records: (30946,)

=== idascore ===


,cid,name,type,notnull,dflt_value,pk
0,0,EmbryoID,VARCHAR,False,None,False
1,1,IDAScore,VARCHAR,False,None,False
2,2,IDATime,VARCHAR,False,None,False
3,3,IDAVersion,VARCHAR,False,None,False
4,4,IDATimestamp,VARCHAR,False,None,False
5,5,_extraction_timestamp,VARCHAR,False,None,False
6,6,_location,VARCHAR,False,None,False
7,7,_run_id,VARCHAR,False,None,False
8,8,_row_hash,VARCHAR,False,None,False


...

,PatientIDx,PatientID,FirstName,LastName,_extraction_timestamp,_location,_run_id,_row_hash,DateOfBirth
0,NEXTGEN_43622.7870662732,1111,first,test,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,ce7eeef44db2e45a9ee560d50c47571b,NaT
1,PC10T4L72760_43623.4196205208,9999,Jane,Doe,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,0e80d8bb37d49d69dcc2db6fdd119e1f,NaT
2,PC10T4L790165_43625.4810629630,62399,Amanda Alves,Pollone,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,ae933c1906afa46920a048bd89f3004f,1983-12-01
3,PC10T4L741253_43626.4449943171,67549,"Silva, Vanessa",L.,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,37ce5a215c60a9ead635c2350594dd11,1980-05-01
4,PC10T4L741253_43626.4518250926,68786,"Silveira,",Gislaine,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,bdfdc5f751721a992f783e59f671a302,1978-11-01


Records: (2654,)

=== treatments ===


,cid,name,type,notnull,dflt_value,pk
0,0,PatientIDx,VARCHAR,False,None,False
1,1,TreatmentName,VARCHAR,False,None,False
2,2,_extraction_timestamp,VARCHAR,False,None,False
3,3,_location,VARCHAR,False,None,False
4,4,_run_id,VARCHAR,False,None,False
5,5,_row_hash,VARCHAR,False,None,False


,PatientIDx,TreatmentName,_extraction_timestamp,_location,_run_id,_row_hash
0,NEXTGEN_43622.7870662732,1icsi,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,547067ba7f7c02d100f6c2017b566648
1,NEXTGEN_43622.7870662732,2022 - 12,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,29e103fd7f34cd3e376b53ffc7f779c3
2,NEXTGEN_43622.7870662732,Test August,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,229cf5bb2472fdb0fc43e68b7081934f
3,PC10T4L72760_43623.4196205208,1,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,a93a119ac31ba224d63babb8dad94fa9
4,PC10T4L72760_43623.4196205208,2,2025-07-15 19:21:41.460065,Vila Mariana,5a1a0509-7c67-43d2-a882-27ef8fcbf469,3285ffa98d446612099c62bf105f9cf9


Records: (3541,)


In [14]:
# df['raw_json'].values